In [1]:
!pip install rank_bm25 --quiet #install BM25
!pip install --no-binary :all: nmslib #install nmslib
!pip install ftfy
!pip install nmslib
!pip install spacy
import os
import pandas as pd
import numpy as np
import pickle
import spacy
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
from rank_bm25 import BM25Okapi
import nmslib
import time

#drive.mount('/content/drive') # if you want to use G Drive
pd.set_option('display.max_colwidth', -1)
plt.style.use('fivethirtyeight')


<ipython-input-1-d1eb7e9c7f30>:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### Loading Pickle files and data files

In [34]:
filename= 'ft_model.sav'
ft_model = pickle.load(open(filename, 'rb'))
df = pd.read_pickle("data.pkl")

### Load document vectors, build index and search:

In [73]:
with open( "weighted_doc_vects.p", "rb" ) as f:
  weighted_doc_vects = pickle.load(f)
# create a random matrix to index
data = np.vstack(weighted_doc_vects)

# initialize a new index, using a HNSW index on Cosine Similarity - can take a couple of mins


index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

In [78]:
# filename= 'index.p'
# index = pickle.load(open(filename, 'rb'))

# querying the index:
input = 'employee'.lower().split()


query = [ft_model[vec] for vec in input]
query = np.mean(query,axis=0)
df_new=[]
t0 = time.time()
ids, distances = index.knnQuery(query, k=20)
t1 = time.time()
print(f'Searched {df.shape[0]} records in {round(t1-t0,4) } seconds \n')
for i,j in zip(ids,distances):
    df_new.append(f'{df.API.values[i]} {df.Label.values[i]} ' )
#     print(round(j,2))
#     df_new.add(df['Label'].values[i])

    

print(df_new)

Searched 6724 records in 0.0004 seconds 

['RingLead Essential Firmographics Website - Online Since ', 'RingLead Essential Contact Website - Online Since ', 'InsideView Website List ', 'RingLead Essential Firmographics Website - Language ', 'RingLead Essential Contact Website - Language ', 'Cognism Person Previous Employer Website - Domain ', 'Lusha Person - Website of Employer ', 'Everstring ES Primary Website ', 'Everstring ES Primary Website ', 'Cognism Person Employer Website - Domain ', 'Cognism Person Previous Employer Finance Detail ', 'Pipl - RingLead Managed Urls Valid Since ', 'Pipl Urls Valid Since ', 'People Data Labs Primary Job Company Website ', 'Cognism Person Previous Employer Alias List ', 'Cognism Person Previous Employer Keywords List ', 'Cognism Person Previous Employer Technology List ', 'Cognism Person Previous Employer Company Name ', 'Cognism Person Previous Employer Social Network Detail ', 'Cognism Person Previous Employer Milestone Detail ']


<ipython-input-78-afd2ada5a46d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  query = [ft_model[vec] for vec in input]


In [79]:
index.saveIndex("Indexdata.bin", save_data=True)